## Let's configure our environment

In [3]:
import sys
!{sys.executable} -m pip install sagemaker -U
!{sys.executable} -m pip install sagemaker-experiments

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.5/671.5 kB 3.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 48.2 MB/s eta 0:00:0000:0100:01
  Created wheel for sagemaker: filename=sagemaker-2.133.0-py2.py3-none-any.whl size=908931 sha256=ed2a9c259a7c0c335c71c76bf00fa4341d47a1ace6cdbd414fe88768d35dcc11
  Stored in directory: /home/ec2-user/.cache/pip/wheels/d4/3b/d6/3b701d2e6779e4bbefc78407bb925429c5b0b07a0126990ee0
Successfully built sagemaker
  Attempting uninstall: botocore
    Found existing installation: botocore 1.24.19
    Uninstalling botocore-1.24.19:
      Successfully uninstalled botocore-1.24.19
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.127.0
    Uninstalling sagemaker-2.127.0:
      Successfully uninstalled sagemaker-2.127.0
ERROR: pip's dependency resolver does not current

## Restart the Kernel and Run from Below

In [1]:
import sagemaker
from sagemaker import get_execution_role

In [2]:
sagemaker_session = sagemaker.Session()

In [3]:
# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

In [4]:
role

'arn:aws:iam::130391109865:role/service-role/AmazonSageMaker-ExecutionRole-20230218T114586'

## Upload the data for training

In [5]:
train_input = sagemaker_session.upload_data("data")

In [7]:
train_input

's3://sagemaker-ap-south-1-130391109865/data'

## Create SageMaker Scikit Estimator

In [9]:
from sagemaker.sklearn.estimator import SKLearn

script_path = 'startup_prediction.py'

sklearn = SKLearn(
    entry_point=script_path,
    instance_type="ml.m4.xlarge",
    framework_version="0.20.0",
    py_version="py3",
    role=role,
    sagemaker_session=sagemaker_session)

## Train SKLearn Estimator on Startup data

In [10]:
sklearn.fit({'train': train_input})

INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2023-02-18-08-35-08-988


2023-02-18 08:35:09 Starting - Starting the training job...
2023-02-18 08:35:35 Starting - Preparing the instances for training......
2023-02-18 08:36:32 Downloading - Downloading input data...
2023-02-18 08:36:52 Training - Downloading the training image...
2023-02-18 08:37:38 Training - Training image download completed. Training in progress...2023-02-18 08:37:52,615 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2023-02-18 08:37:52,618 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-02-18 08:37:52,628 sagemaker_sklearn_container.training INFO     Invoking user training script.
2023-02-18 08:37:52,825 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-02-18 08:37:52,838 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-02-18 08:37:52,852 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-02-18 08

## Deploy the model

In [11]:
deployment = sklearn.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2023-02-18-08-44-32-321
INFO:sagemaker:Creating endpoint-config with name sagemaker-scikit-learn-2023-02-18-08-44-32-321
INFO:sagemaker:Creating endpoint with name sagemaker-scikit-learn-2023-02-18-08-44-32-321


------!

In [12]:
deployment.endpoint

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


'sagemaker-scikit-learn-2023-02-18-08-44-32-321'

#### Prediction on the endpoint

In [14]:
deployment.predict([[1,0,50000,25000,40000]]).round(2)

array([82554.76])